In [1]:
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.linear_model import LogisticRegression #logistic regression
from sklearn import svm #support vector Machine
from sklearn.ensemble import RandomForestClassifier #Random Forest
from sklearn.model_selection import train_test_split #training and testing data split
from sklearn import metrics #accuracy measure
from sklearn.metrics import confusion_matrix #for confusion matrix
from sklearn.metrics import f1_score
from time import time
from operator import itemgetter
from scipy.stats import randint
from sklearn.tree import DecisionTreeClassifier
from sklearn.tree import export_graphviz
from sklearn.grid_search import GridSearchCV
from sklearn.grid_search import RandomizedSearchCV
from sklearn.cross_validation import  cross_val_score
from __future__ import print_function
import os
import subprocess
from time import time
from operator import itemgetter
from scipy.stats import randint
import pandas as pd
import numpy as np
print(__doc__)
import numpy as np
from time import time
from scipy.stats import randint as sp_randint
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV
from sklearn.datasets import load_digits
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import export_graphviz
from sklearn.grid_search import GridSearchCV
from sklearn.grid_search import RandomizedSearchCV
from sklearn.cross_validation import  cross_val_score
from pprint import pprint


filename = 'df_imputed__OHE_cat_std_num.pkl'
df = pd.read_pickle(filename)

X = df[df.columns[:-1]]
y = pd.Series(df['heartdisease'])


train_X,test_X,train_y,test_y =train_test_split(X,y,test_size=0.33,shuffle = True, random_state=45)

Automatically created module for IPython interactive environment


/anaconda3/lib/python3.6/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)
/anaconda3/lib/python3.6/site-packages/sklearn/grid_search.py:42: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. This module will be removed in 0.20.
  DeprecationWarning)


In [3]:
def report(grid_scores, n_top=3):
    """Report top n_top parameters settings, default n_top=3.

    Args
    ----
    grid_scores -- output from grid or random search
    n_top -- how many to report, of top models

    Returns
    -------
    top_params -- [dict] top parameter settings found in
                  search
    """
    top_scores = sorted(grid_scores,
                        key=itemgetter(1),
                        reverse=True)[:n_top]
    for i, score in enumerate(top_scores):
        print("Model with rank: {0}".format(i + 1))
        print(("Mean validation score: "
               "{0:.3f} (std: {1:.3f})").format(
               score.mean_validation_score,
               np.std(score.cv_validation_scores)))
        print("Parameters: {0}".format(score.parameters))
        print("")

    return top_scores[0].parameters

In [4]:
def run_gridsearch(X, y, clf, param_grid, cv=5):
    """Run a grid search for best Decision Tree parameters.

    Args
    ----
    X -- features
    y -- targets (classes)
    cf -- scikit-learn Decision Tree
    param_grid -- [dict] parameter settings to test
    cv -- fold of cross-validation, default 5

    Returns
    -------
    top_params -- [dict] from report()
    """
    grid_search = GridSearchCV(clf,
                               param_grid=param_grid,
                               cv=cv)
    start = time()
    grid_search.fit(X, y)

    print(("\nGridSearchCV took {:.2f} "
           "seconds for {:d} candidate "
           "parameter settings.").format(time() - start,
                len(grid_search.grid_scores_)))

    top_params = report(grid_search.grid_scores_, 3)
    return  top_params

In [5]:
def run_randomsearch(X, y, clf, para_dist, cv=5,
                     n_iter_search=100):
    """Run a random search for best Decision Tree parameters.

    Args
    ----
    X -- features
    y -- targets (classes)
    cf -- scikit-learn Decision Tree
    param_dist -- [dict] list, distributions of parameters
                  to sample
    cv -- fold of cross-validation, default 5
    n_iter_search -- number of random parameter sets to try,
                     default 20.

    Returns
    -------
    top_params -- [dict] from report()
    """
    random_search = RandomizedSearchCV(clf,
                        param_distributions=param_dist,
                        n_iter=n_iter_search)

    start = time()
    random_search.fit(X, y)
    print(("\nRandomizedSearchCV took {:.2f} seconds "
           "for {:d} candidates parameter "
           "settings.").format((time() - start),
                               n_iter_search))

    top_params = report(random_search.grid_scores_, 3)
    return  top_params

In [7]:
print("-- 10-fold cross-validation "
      "[using setup from previous post]")

rfc_old = RandomForestClassifier(n_estimators=20)
rfc_old.fit(X, y)
scores = cross_val_score(rfc_old, X, y, cv=10)
print("mean: {:.3f} (std: {:.3f})".format(scores.mean(),
                                          scores.std()),
                                          end="\n\n" )

-- 10-fold cross-validation [using setup from previous post]
mean: 0.762 (std: 0.068)



In [8]:
print("-- Random Parameter Search via 3-fold CV")

# Number of trees in random forest
n_estimators = [int(x) for x in np.linspace(start = 200, stop = 2000, num = 10)]
# Number of features to consider at every split
max_features = ['auto', 'sqrt']
# Maximum number of levels in tree
max_depth = [int(x) for x in np.linspace(10, 110, num = 11)]
max_depth.append(None)
# Minimum number of samples required to split a node
min_samples_split = [2, 5, 10]
# Minimum number of samples required at each leaf node
min_samples_leaf = [1, 2, 4]
# Method of selecting samples for training each tree
bootstrap = [True, False]
# Create the random grid
param_dist = {'n_estimators': n_estimators,
               'max_features': max_features,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf,
               'bootstrap': bootstrap}
pprint(param_dist)




rfc = RandomForestClassifier()
ts_rs = run_randomsearch(X, y, rfc, param_dist, cv=3, n_iter_search=100)

-- Random Parameter Search via 3-fold CV
{'bootstrap': [True, False],
 'max_depth': [10, 20, 30, 40, 50, 60, 70, 80, 90, 100, 110, None],
 'max_features': ['auto', 'sqrt'],
 'min_samples_leaf': [1, 2, 4],
 'min_samples_split': [2, 5, 10],
 'n_estimators': [200, 400, 600, 800, 1000, 1200, 1400, 1600, 1800, 2000]}

RandomizedSearchCV took 580.86 seconds for 100 candidates parameter settings.
Model with rank: 1
Mean validation score: 0.788 (std: 0.037)
Parameters: {'n_estimators': 200, 'min_samples_split': 2, 'min_samples_leaf': 4, 'max_features': 'sqrt', 'max_depth': 110, 'bootstrap': True}

Model with rank: 2
Mean validation score: 0.785 (std: 0.042)
Parameters: {'n_estimators': 800, 'min_samples_split': 5, 'min_samples_leaf': 4, 'max_features': 'sqrt', 'max_depth': 40, 'bootstrap': True}

Model with rank: 3
Mean validation score: 0.785 (std: 0.042)
Parameters: {'n_estimators': 1200, 'min_samples_split': 5, 'min_samples_leaf': 4, 'max_features': 'sqrt', 'max_depth': 30, 'bootstrap': Tru

In [33]:
print("-- Random Parameter Search via 5-fold CV")

# Number of trees in random forest
n_estimators = [int(x) for x in np.linspace(start = 200, stop = 2000, num = 10)]
# Number of features to consider at every split
max_features = ['auto', 'sqrt']
# Maximum number of levels in tree
max_depth = [int(x) for x in np.linspace(10, 110, num = 11)]
max_depth.append(None)
# Minimum number of samples required to split a node
min_samples_split = [2, 5, 10]
# Minimum number of samples required at each leaf node
min_samples_leaf = [1, 2, 4]
# Method of selecting samples for training each tree
bootstrap = [True, False]
# Create the random grid
param_dist = {'n_estimators': n_estimators,
               'max_features': max_features,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf,
               'bootstrap': bootstrap}
pprint(param_dist)




rfc = RandomForestClassifier()
ts_rs = run_randomsearch(X, y, rfc, param_dist, cv=5, n_iter_search=100)

-- Random Parameter Search via 5-fold CV
{'bootstrap': [True, False],
 'max_depth': [10, 20, 30, 40, 50, 60, 70, 80, 90, 100, 110, None],
 'max_features': ['auto', 'sqrt'],
 'min_samples_leaf': [1, 2, 4],
 'min_samples_split': [2, 5, 10],
 'n_estimators': [200, 400, 600, 800, 1000, 1200, 1400, 1600, 1800, 2000]}

RandomizedSearchCV took 483.05 seconds for 100 candidates parameter settings.
Model with rank: 1
Mean validation score: 0.782 (std: 0.033)
Parameters: {'n_estimators': 200, 'min_samples_split': 2, 'min_samples_leaf': 4, 'max_features': 'auto', 'max_depth': 20, 'bootstrap': True}

Model with rank: 2
Mean validation score: 0.780 (std: 0.033)
Parameters: {'n_estimators': 400, 'min_samples_split': 10, 'min_samples_leaf': 4, 'max_features': 'sqrt', 'max_depth': 20, 'bootstrap': True}

Model with rank: 3
Mean validation score: 0.775 (std: 0.034)
Parameters: {'n_estimators': 800, 'min_samples_split': 10, 'min_samples_leaf': 4, 'max_features': 'auto', 'max_depth': 70, 'bootstrap': Tru

In [ ]:
print("\n-- Best Parameters:")
for k, v in ts_rs.items():
    print("parameters: {:<20s} setting: {}".format(k, v))

In [9]:
print("-- Random Parameter Search via 10-fold CV")

# Number of trees in random forest
n_estimators = [int(x) for x in np.linspace(start = 200, stop = 2000, num = 10)]
# Number of features to consider at every split
max_features = ['auto', 'sqrt']
# Maximum number of levels in tree
max_depth = [int(x) for x in np.linspace(10, 110, num = 11)]
max_depth.append(None)
# Minimum number of samples required to split a node
min_samples_split = [2, 5, 10]
# Minimum number of samples required at each leaf node
min_samples_leaf = [1, 2, 4]
# Method of selecting samples for training each tree
bootstrap = [True, False]
# Create the random grid
param_dist = {'n_estimators': n_estimators,
               'max_features': max_features,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf,
               'bootstrap': bootstrap}
pprint(param_dist)




rfc = RandomForestClassifier()
ts_rs = run_randomsearch(X, y, rfc, param_dist, cv=10, n_iter_search=100)

-- Random Parameter Search via 10-fold CV
{'bootstrap': [True, False],
 'max_depth': [10, 20, 30, 40, 50, 60, 70, 80, 90, 100, 110, None],
 'max_features': ['auto', 'sqrt'],
 'min_samples_leaf': [1, 2, 4],
 'min_samples_split': [2, 5, 10],
 'n_estimators': [200, 400, 600, 800, 1000, 1200, 1400, 1600, 1800, 2000]}

RandomizedSearchCV took 610.26 seconds for 100 candidates parameter settings.
Model with rank: 1
Mean validation score: 0.786 (std: 0.039)
Parameters: {'n_estimators': 1200, 'min_samples_split': 10, 'min_samples_leaf': 4, 'max_features': 'sqrt', 'max_depth': 70, 'bootstrap': True}

Model with rank: 2
Mean validation score: 0.785 (std: 0.040)
Parameters: {'n_estimators': 1400, 'min_samples_split': 5, 'min_samples_leaf': 4, 'max_features': 'auto', 'max_depth': 70, 'bootstrap': True}

Model with rank: 3
Mean validation score: 0.785 (std: 0.035)
Parameters: {'n_estimators': 200, 'min_samples_split': 5, 'min_samples_leaf': 4, 'max_features': 'sqrt', 'max_depth': 70, 'bootstrap': T

In [35]:
print("\n-- Best Parameters:")
for k, v in ts_rs.items():
    print("parameters: {:<20s} setting: {}".format(k, v))


-- Best Parameters:
parameters: n_estimators         setting: 400
parameters: min_samples_split    setting: 5
parameters: min_samples_leaf     setting: 4
parameters: max_features         setting: auto
parameters: max_depth            setting: 60
parameters: bootstrap            setting: True


In [37]:
# test the retuned best parameters
print("\n\n-- Testing best parameters [Random]...")
dt_ts_rs = RandomForestClassifier(**ts_rs)
scores = cross_val_score(dt_ts_rs, X, y, cv=10)
print("mean: {:.3f} (std: {:.3f})".format(scores.mean(),
                                          scores.std()),
                                          end="\n\n" )



-- Testing best parameters [Random]...
mean: 0.783 (std: 0.072)



In [56]:
# test the retuned best parameters
print("\n\n-- Testing best parameters [Random] CV=3 ...")
dt_ts_rs = RandomForestClassifier(n_estimators= 400,
               max_features= 'sqrt',
               max_depth= 30,
               min_samples_split= 2,
               min_samples_leaf= 4,
               bootstrap= True)

scores_3fold = cross_val_score(dt_ts_rs, X, y, cv=10)
print("mean: {:.3f} (std: {:.3f})".format(scores_3fold.mean(),
                                          scores_3fold.std()),
                                          end="\n\n" )

print("\n\n-- Testing best parameters [Random] CV=5 ...")
dt_ts_rs = RandomForestClassifier(n_estimators= 200,
               max_features= 'auto',
               max_depth= 20,
               min_samples_split= 2,
               min_samples_leaf= 4,
               bootstrap= True)

scores_5fold = cross_val_score(dt_ts_rs, X, y, cv=10)
print("mean: {:.3f} (std: {:.3f})".format(scores_5fold.mean(),
                                          scores_5fold.std()),
                                          end="\n\n" )

print("\n\n-- Testing best parameters [Random] CV=10 ...")

dt_ts_rs = RandomForestClassifier(n_estimators= 400,
               max_features= 'auto',
               max_depth= 60,
               min_samples_split= 5,
               min_samples_leaf= 4,
               bootstrap= True)

scores_10fold = cross_val_score(dt_ts_rs, X, y, cv=10)
print("mean: {:.3f} (std: {:.3f})".format(scores_10fold.mean(),
                                          scores_10fold.std()),
                                          end="\n\n" )




-- Testing best parameters [Random] CV=3 ...
mean: 0.791 (std: 0.068)



-- Testing best parameters [Random] CV=5 ...
mean: 0.781 (std: 0.072)



-- Testing best parameters [Random] CV=10 ...
mean: 0.778 (std: 0.075)



In [75]:
def evaluate(model, test_features, test_labels):
    predictions = model.predict(test_features)
    
    errors = abs(predictions - test_labels)
    pprint('this is errors ' + str(errors))
    pprint('this is predictions ' + str(predictions))
    mape = 100 * np.mean(abs(errors) / abs(test_labels))
    
    accuracy = 100 - mape
    print('Model Performance')
    print('Average Error: {:0.4f} degrees.'.format(np.mean(errors)))
    print('Accuracy = {:0.2f}%.'.format(accuracy))
    
    return accuracy




In [80]:
rfc_best = RandomForestClassifier(n_estimators= 400,
               max_features= 'sqrt',
               max_depth= 30,
               min_samples_split= 2,
               min_samples_leaf= 4,
               bootstrap= True)
rfc_base = RandomForestClassifier(n_estimators= 200, random_state = 42)



rfc_base.fit(train_X, train_y)
rfc_best.fit(train_X, train_y)
prediction_base=rfc_base.predict(test_X)
prediction_best=rfc_best.predict(test_X)


print('Test Accuracy for RF base is ',metrics.accuracy_score(prediction_base,test_y))
print('Test Accuracy for RF best is ',metrics.accuracy_score(prediction_best,test_y))


prediction_base_train=rfc_base.predict(train_X)
prediction_best_train=rfc_best.predict(train_X)
print('Training Accuracy for RF base is ',metrics.accuracy_score(train_y,prediction_base_train))
print('Training Accuracy for RF best is ',metrics.accuracy_score(train_y,prediction_best_train))





Test Accuracy for RF base is  0.8256578947368421
Test Accuracy for RF best is  0.819078947368421
Training Accuracy for RF base is  1.0
Training Accuracy for RF best is  0.8668831168831169
